<a href="https://colab.research.google.com/github/Supriya772/Supriya/blob/main/CoaProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opencv-python SimpleITK


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 17.4 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
import multiprocessing
import time

def compute_histogram(image, start, end, bins):
    histogram, _ = np.histogram(image[start:end].ravel(), bins=bins, range=[0, 256])
    return histogram

def otsu_threshold(image):
    flattened_image = image.ravel()
    bins = 256
    half_height = image.shape[0] // 2

    start_time = time.time()

    with multiprocessing.Pool(processes=2) as pool:
        histograms = pool.starmap(compute_histogram, [(flattened_image, 0, half_height * image.shape[1], bins),
                                                      (flattened_image, half_height * image.shape[1], None, bins)])

    total_histogram = np.sum(histograms, axis=0)
    probabilities = total_histogram / np.sum(total_histogram)
    cumulative_distribution = np.cumsum(probabilities)
    thresholds = np.arange(0, 256)
    variance_between_classes = (cumulative_distribution * (1.0 - cumulative_distribution))**2 / probabilities
    optimal_threshold = np.argmax(variance_between_classes)

    end_time = time.time()

    _, segmented_image = cv2.threshold(image, optimal_threshold, 255, cv2.THRESH_BINARY)

    return segmented_image, end_time - start_time

if __name__ == "__main__":
    # Read the image
    image_path = "/content/brain.jpeg"
    original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Serial segmentation
    serial_start_time = time.time()
    serial_result, serial_time = otsu_threshold(original_image)
    serial_end_time = time.time()

    cv2.imwrite("serial_segmentation_result.jpg", serial_result)

    # Parallel segmentation
    parallel_start_time = time.time()
    parallel_result, parallel_time = otsu_threshold(original_image)
    parallel_end_time = time.time()

    cv2.imwrite("parallel_segmentation_result.jpg", parallel_result)

    # Calculate speedup
    speedup_result = serial_time / parallel_time

    print(f"Serial segmentation time: {serial_time} seconds")
    print(f"Parallel segmentation time: {parallel_time} seconds")
    print(f"Speedup: {speedup_result:.2f}")


Serial segmentation time: 0.06487274169921875 seconds
Parallel segmentation time: 0.05656242370605469 seconds
Speedup: 1.15
